In [1]:
if "running_all" not in globals():
    from ipywidgets import widgets
    matrices = ["bcsstk18", "cbuckle", "ct20stif", "raefsky4", "vanbody"]
    b = widgets.Button(description="Run over all matrices", button_style="success")
    output = widgets.Output()

    display(b, output)

    def run_over_all_matrices(button):
        global running_all
        global matrix
        running_all = True
        with output:
            for matrix in matrices:
                print(f"Running {matrix}...")
                %run ./protection_vis.ipynb # will output at this cell rather than later
            print("Finished!")
    b.on_click(run_over_all_matrices)

Button(button_style='success', description='Run over all matrices', style=ButtonStyle())

Output()

In [ ]:
if "running_all" not in globals():
    matrix = "cbuckle"  # manually set to run over one matrix

In [ ]:
from scipy.sparse.linalg import norm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))
from io_utils import load_matrices_from_dir, load_models

In [ ]:
mats = load_matrices_from_dir("../matrices/raw", subset=[matrix])
mat = list(mats.values())[0]
mat

In [ ]:
mat_errorfree_iters = {
    "bcsstk18": 432,
    "cbuckle": 341,
    "ct20stif": 364,
    "raefsky4": 79,
    "vanbody": 491,
}
errorfree_iterations = mat_errorfree_iters[matrix]

In [ ]:
chosen_models = {
    "bcsstk18": "KNeighborsRegressor",
    "cbuckle": "RandomForestRegressor",
    "ct20stif": "XGBRegressor",
    "raefsky4": "RandomForestRegressor",
    "vanbody": "KNeighborsRegressor",
}
model_path = f"./models/{matrix}/best_{chosen_models[matrix]}.pkl"
model = load_models([model_path])[0]
model

In [ ]:
pos_2norms = {pos: norm(mat.getrow(pos)) for pos in range(mat.shape[0])}
iter_norm_pairs = np.array([[i, pos_2norms[pos], pos] for pos in range(mat.shape[0])
                              for i in range(errorfree_iterations)])
df = pd.DataFrame(iter_norm_pairs, columns=["i", "2norm", "rowid"])
df["output"] = model.predict(df[["i", "2norm"]].to_numpy())
df.head()

In [ ]:
max_2norm = df["2norm"].max()
spaced_2norms = np.geomspace(1e-6, max_2norm, num=1000)
iter_norm_pairs_spaced = np.array([[i, n, p] for p, n in enumerate(spaced_2norms)
                                       for i in range(errorfree_iterations)])
df_spaced = pd.DataFrame(iter_norm_pairs_spaced, columns=["i", "2norm", "rowid"])
df_spaced["output"] = model.predict(df_spaced[["i", "2norm"]].to_numpy())
df_spaced.head()

In [ ]:
df_spaced_piv = df_spaced.pivot(index=["2norm", "rowid"], columns="i", values="output")

protections = df_spaced_piv.to_numpy()
df_spaced_piv.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)

ax = sns.heatmap(df_spaced_piv, vmin=1, vmax=100)

plt.title("Predicted Outputs by Iteration and Row 2-norm")
plt.xlabel("Iteration Number")
plt.ylabel("Corresponding Row 2-norm (log scale)")
plt.savefig(f"./figures/{matrix}/iter_r2n_output.png")
plt.show()

In [ ]:
df_piv = df.pivot(index=["rowid"], columns="i", values="output")

protections = df_piv.to_numpy()
df_piv.head()

In [ ]:
plt.gcf().set_size_inches(8, 8)
plt.gcf().set_dpi(100)
sns.heatmap(protections, vmin=0, vmax=100)
plt.title("Protection Scheme")
plt.xlabel("Iteration Number")
plt.ylabel("Row Index")
plt.savefig(f"./figures/{matrix}/iter_row_output.png")
plt.show()